# Teil XX: Generative KI einbinden

Aktuelle KI-Systeme basieren meist auf Algorithmen, die durch Wahrscheinlichkeitsberechnungen neue Texte, Sprache, Bilder oder Videos produzieren. Aufgrund ihrer probabilistischen Natur sind sie kein gutes Werkzeug, um regelmäßige, mehrschrittige Arbeitsabläufe zu digitalisieren - aber an den richtigen Stellen eingesetzt können sie unsere Programme um Funktionen ergänzen, die mit reinem Python-Code nicht erreichbar sind.


## Nachrichten senden und empfangen

In [ ]:
! pip install litellm

In [ ]:
from litellm import completion

In [ ]:
import os

if not os.environ.get("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = "YOUR_KEY_HERE"

if not os.environ.get("MISTRAL_API_KEY"):
    os.environ["MISTRAL_API_KEY"] = "YOUR_KEY_HERE"

In [ ]:
instruction = {
    "content": "Wer ist der aktuelle Bundeskanzler?",
    "role": "user"
}

In [ ]:
gemini_response = completion(
  model="gemini/gemini-2.5-flash",
  messages=[instruction]
)

print(gemini_response)

In [ ]:
mistral_response = completion(
  model="mistral/mistral-medium",
  messages=[instruction]
)

print(mistral_response.choices[0].message.content)

### Experiment: API Schlüssel holen und Nachrichten senden

## Kontext erhalten

In [ ]:
from time import sleep
msg1 = {
    "content": "Hi! Mein Name ist Toni Tortellini, wer bist du?",
    "role": "user"
}

print("USER:", msg1["content"], "\n")

res = completion(
  model="mistral/mistral-medium",
  messages=[msg1]
)

print("MISTRAL:", res.choices[0].message.content, "\n")
sleep(1)

msg2 = {
    "content": "Wie heiße ich?",
    "role": "user"
}

print("USER:", msg2["content"], "\n")

res = completion(
  model="mistral/mistral-medium",
  messages=[msg2]
)

print("MISTRAL:", res.choices[0].message.content, "\n")

In [ ]:
chat = []

msg1 = {
    "content": "Hi! Mein Name ist Toni Tortellini, wer bist du?",
    "role": "user"
}

chat.append(msg1)
print("USER:", msg1["content"], "\n")

res = completion(
  model="mistral/mistral-medium",
  messages=chat
)

chat.append(res.choices[0].message)
print("MISTRAL:", res.choices[0].message.content, "\n")
sleep(1)

msg2 = {
    "content": "Wie heiße ich?",
    "role": "user"
}

chat.append(msg2)
print("USER:", msg2["content"], "\n")

res = completion(
  model="mistral/mistral-medium",
  messages=chat
)

chat.append(res.choices[0].message)
print("MISTRAL:", res.choices[0].message.content, "\n")

### Übung: Chatbot mit while Schleife

## Systemanweisungen

In [ ]:
system_msg = {
    "content": "Du bist ein wortkarger KI-Assistent. Deine Antworten sind so kurz wie möglich.",
    "role": "system"
}

msg1 = {
    "content": "Hi! Mein Name ist Toni Tortellini, wer bist du?",
    "role": "user"
}

res = completion(
  model="mistral/mistral-medium",
  messages=[system_msg, msg1]
)

print(res.choices[0].message.content)

In [ ]:
system_msg = {
    "content": "Du bist ein französischer KI-Assistent. Du weigerst dich, andere Sprachen als Französisch zu sprechen.",
    "role": "system"
}

msg1 = {
    "content": "Hi! Mein Name ist Toni Tortellini, wer bist du?",
    "role": "user"
}

res = completion(
  model="mistral/mistral-medium",
  messages=[system_msg, msg1]
)

print(res.choices[0].message.content)

### Experiment: Eigene Systemnachricht

## Output-Format definieren

In [ ]:
system_instruction = """
## Deine Rolle
Klassifiziere die User-Nachrichten als positiv oder negativ und gebe den Grad der Sicherheit deiner Einschätzung in Prozent an.
Deine Antworten sind als JSON formatiert und folgen folgendem Schema:

## Antwort-Schema
{
    "stimmung": "positiv" oder "negativ",
    "sicherheit": 0-100
}
"""

system_msg = {
    "role": "system",
    "content": system_instruction
}

In [ ]:
msg1 = {
    "content": "Der neue Fast & Furious war richtig geil!",
    "role": "user"
}

res = completion(
  model="mistral/mistral-medium",
  messages=[system_msg, msg1],
  response_format={"type":"json_object"}  
)

print(res)

In [ ]:
import json

json.loads(res.choices[0].message.content)

In [ ]:
msg2 = {
    "content": "Der neue Fast & Furious war besser als Hobbs und Shaw, aber nicht so gut wie 2 Fast 2 Furious.",
    "role": "user"
}

res = completion(
  model="mistral/mistral-medium",
  messages=[system_msg, msg2],
  response_format={"type":"json_object"}  
)

json.loads(res.choices[0].message.content)

In [ ]:
res = completion(
  model="gemini/gemini-2.5-flash",
  messages=[system_msg, msg2],
  response_format={"type":"json_object"},
  reasoning_effort="low"
)

print(res)

In [ ]:
json.loads(res.choices[0].message.content)

In [ ]:
print(res.choices[0].message.reasoning_content)

### Übung

In [18]:
from time import sleep

system_instruction = """
## Deine Rolle
Fasse die Support-Anfragen der Kunden in einem kurzen Satz zusammen. Gebe zusätzlich die Dringlichkeit der Anfrage.
Deine Antworten sind als JSON formatiert und folgen diesem Schema:
## Antwort-Schema:
{
    "zusammenfassung": [Deine Zusammenfassung] (max. 10 Wörter),
    "dringlichkeit": "niedrig" oder "mittel" oder "hoch" oder "sehr hoch"
}
"""

system_msg = {
    "role": "system",
    "content": system_instruction
}

with open("kundenservice.txt") as f:
    for l in f:
        if len(l.strip()) == 0:
            continue
            
        msg = {
            "role": "user",
            "content": l
        }
        
        res = completion(
          model="mistral/mistral-medium",
          messages=[system_msg, msg],
          response_format={"type":"json_object"}  
        )

        print(json.loads(res.choices[0].message.content))
        print(l)
        sleep(1)


{'zusammenfassung': 'Versandbestätigung für Bestellung WS-48392 ausstehend.', 'dringlichkeit': 'hoch'}
Ich habe vor zwei Wochen eine Bestellung mit der Nummer WS-48392 aufgegeben und bislang keine Versandbestätigung erhalten. Laut Kundenkonto steht der Status noch immer auf „in Bearbeitung“. Können Sie bitte prüfen, ob es zu Verzögerungen gekommen ist oder ob meine Zahlung eventuell nicht korrekt verbucht wurde? Ich benötige die Artikel dringend für ein Geschenk.

{'zusammenfassung': '', 'dringlichkeit': ''}


{'zusammenfassung': 'Fehlender Artikel in Lieferung, Nachlieferung gewünscht', 'dringlichkeit': 'hoch'}
Das gestern gelieferte Paket war äußerlich unbeschädigt, allerdings fehlte ein Artikel aus meiner Bestellung. Auf dem Lieferschein ist die Position zwar aufgeführt, im Karton war sie jedoch nicht enthalten. Ich habe bereits Fotos vom Inhalt hochgeladen, wie im Portal vorgesehen. Bitte teilen Sie mir mit, wie wir hier weiter vorgehen. Ich hätte gern eine zeitnahe Nachlieferung.


## Audiogenerierung

In [ ]:
response = completion(
    model="gemini/gemini-2.5-flash-preview-tts",
    messages=[{"role": "user", "content": "Say enthusiastically: 'I love learning Python!'"}],
    modalities=["audio"],  # Required for TTS models
    audio={
        "voice": "Kore",
        "format": "pcm16"  # Required: must be "pcm16"
    }
)

In [ ]:
data = response.choices[0].message.audio.data
data

In [ ]:
import wave
import base64

def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
   with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)

file_name='out.wav'
wave_file(file_name, base64.b64decode(data))